In [13]:
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], True)



import numpy as np
import pandas as pd
import seaborn as sb
sb.set_style("dark")
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

train_df_org = pd.read_csv('kaggle/input/digit-recognizer/train.csv',  nrows=100)

mnist_train_data = train_df_org.loc[:, "pixel0":]
mnist_train_label = train_df_org.loc[:, "label"]

In [14]:


import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras import metrics
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Lambda, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPool2D, AvgPool2D
from tensorflow.keras.optimizers import Adadelta
from keras.utils.np_utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import LearningRateScheduler

In [15]:
# Converting dataframe into arrays
mnist_train_data = np.array(mnist_train_data)
mnist_train_label = np.array(mnist_train_label)
# Reshaping the input shapes to get it in the shape which the model expects to recieve later.
mnist_train_data = mnist_train_data.reshape(mnist_train_data.shape[0], 28, 28, 1)
print(mnist_train_data.shape, mnist_train_label.shape)

(100, 28, 28, 1) (100,)


In [16]:
nclasses = mnist_train_label.max() - mnist_train_label.min() + 1
mnist_train_label = to_categorical(mnist_train_label, num_classes = nclasses)
print("Shape of ytrain after encoding: ", mnist_train_label.shape)


Shape of ytrain after encoding:  (100, 10)


In [17]:
def build_model(input_shape=(28, 28, 1)):
    model = Sequential()
    model.add(Conv2D(32, kernel_size = 3, activation='relu', input_shape = input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(32, kernel_size = 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(64, kernel_size = 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(128, kernel_size = 4, activation='relu'))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(10, activation='softmax'))
    return model


def compile_model(model, optimizer='adam', loss='categorical_crossentropy'):
    model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])


def train_model(model, train, test, epochs, split):
    history = model.fit(train, test, shuffle=True, epochs=epochs, validation_split=split)
    return history

In [18]:
cnn_model = build_model((28, 28, 1))
compile_model(cnn_model, 'adam', 'categorical_crossentropy')

# train the model for as many epochs as you want but I found training it above 80 will not help us and eventually increase overfitting.
model_history = train_model(cnn_model, mnist_train_data, mnist_train_label, 20, 0.2)

Epoch 1/20
3/3 [==============================] - 0s 105ms/step - loss: 3.3097 - accuracy: 0.1000 - val_loss: 2.6149 - val_accuracy: 0.1000
Epoch 2/20
3/3 [==============================] - 0s 59ms/step - loss: 2.1801 - accuracy: 0.3375 - val_loss: 2.4007 - val_accuracy: 0.1000
Epoch 3/20
3/3 [==============================] - 0s 59ms/step - loss: 1.2773 - accuracy: 0.5500 - val_loss: 2.3569 - val_accuracy: 0.0500
Epoch 4/20
3/3 [==============================] - 0s 59ms/step - loss: 1.1653 - accuracy: 0.6000 - val_loss: 2.3206 - val_accuracy: 0.0500
Epoch 5/20
3/3 [==============================] - 0s 60ms/step - loss: 0.9713 - accuracy: 0.6125 - val_loss: 2.3101 - val_accuracy: 0.0500
Epoch 6/20
3/3 [==============================] - 0s 115ms/step - loss: 0.7627 - accuracy: 0.7250 - val_loss: 2.3008 - val_accuracy: 0.0500
Epoch 7/20
3/3 [==============================] - 0s 59ms/step - loss: 0.4547 - accuracy: 0.8750 - val_loss: 2.2996 - val_accuracy: 0.0500
Epoch 8/20
3/3 [=========